In [ ]:
#pip uninstall python-telegram-bot
!pip install python-telegram-bot==13.15


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.4/497.4 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.4 MB/s eta 0:00:00
  Created wheel for tornado: filename=tornado-6.1-cp311-cp311-linux_x86_64.whl size=422003 sha256=3e9a6f3beff3ecb45262df1e468446429548364a4d0c9ea9abe719f6ddadb337
  Stored in directory: /root/.cache/pip/wheels/f2/59/06/a9c85c7b17ec0fc9b1e2ae0c59e3d39255d5c0a38492e33fea
Successfully built tornado
  Attempting uninstall: tornado
    Found existing installation: tornado 6.4.2
    Uninstalling tornado-6.4.2:
      Successfully uninstalled tornado-6.4.2
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.5.1
    Uninstalling cachetools-5.5.1:
      Successfully uninstalled cachetools-5.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are 

In [ ]:
pip install openai


In [ ]:
import logging
import requests
import os
import openai
from openai import OpenAI
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler)
from telegram import ReplyKeyboardMarkup, ParseMode

# API KEYS
GOOGLE_API_KEY = "test"
TELEGRAM_BOT_TOKEN1 ="test"
OPENAI_API_KEY = "test"
openai.api_key = OPENAI_API_KEY
client = OpenAI(
    api_key=openai.api_key)

#  LOGGING SETUP
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# CONVERSATION STATES
NUMBER_OF_PEOPLE, GET_NAMES, COUNTRY, CITY, ACTIVITY, TIME_SPENT, TIME_OF_DAY, BUDGET, AGES, CONFIRMATION = range(10)

# HANDLERS
def start(update, context):
    """Starts the conversation."""
    update.message.reply_text(
        "Hello! I'm your hangout planner bot.\n"
        "How many people are going out together?\n"
        "Type /cancel at any time to stop."
    )
    return NUMBER_OF_PEOPLE

def get_number_of_people(update, context):
    text = update.message.text.strip()
    if not text.isdigit():
        update.message.reply_text("Please enter a valid number.")
        return NUMBER_OF_PEOPLE

    num_people = int(text)
    context.user_data['num_people'] = num_people
    context.user_data['names'] = []

    update.message.reply_text(
        f"Great! We have {num_people} people.\n"
        "Please tell me the name of person 1."
    )
    return GET_NAMES

def get_names(update, context):
    name = update.message.text.strip()
    context.user_data['names'].append(name)

    num_people = context.user_data['num_people']
    current_count = len(context.user_data['names'])

    if current_count < num_people:
        update.message.reply_text(
            f"Got it. Please tell me the name of person {current_count + 1}."
        )
        return GET_NAMES
    else:
        update.message.reply_text("All names recorded! Now, which country are you in?")
        return COUNTRY

def get_country(update, context):
    country = update.message.text.strip()
    context.user_data['country'] = country
    update.message.reply_text("Nice! Which city are you in?")
    return CITY

def get_city(update, context):
    city = update.message.text.strip()
    context.user_data['city'] = city
    update.message.reply_text("What activities do you all like to do?")
    return ACTIVITY

def get_activity(update, context):
    activity = update.message.text.strip()
    context.user_data['activity'] = activity
    update.message.reply_text("How long do you want to spend on your outing?")
    return TIME_SPENT

def get_time_spent(update, context):
    time_spent = update.message.text.strip()
    context.user_data['time_spent'] = time_spent

    reply_keyboard = [['Morning', 'Afternoon', 'Evening', 'Night']]
    update.message.reply_text(
        "Which time of day do you prefer?",
        reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)
    )
    return TIME_OF_DAY

def get_time_of_day(update, context):
    time_of_day = update.message.text.strip()
    context.user_data['time_of_day'] = time_of_day
    update.message.reply_text("What's your budget?")
    return BUDGET

def get_budget(update, context):
    budget = update.message.text.strip()
    context.user_data['budget'] = budget
    update.message.reply_text("Could you share the ages of each person (comma-separated)?")
    return AGES

def get_ages(update, context):
    ages_text = update.message.text.strip()
    ages_list = [age.strip() for age in ages_text.split(',')]
    context.user_data['ages'] = ages_list

    summary = (
        f"Here are the details I have:\n"
        f"- Number of people: {context.user_data['num_people']}\n"
        f"- Names: {', '.join(context.user_data['names'])}\n"
        f"- Ages: {', '.join(ages_list)}\n"
        f"- Location: {context.user_data['city']}, {context.user_data['country']}\n"
        f"- Activity preference: {context.user_data['activity']}\n"
        f"- Time to spend: {context.user_data['time_spent']}\n"
        f"- Preferred time of day: {context.user_data['time_of_day']}\n"
        f"- Budget: {context.user_data['budget']}\n\n"
        "Is this correct? Type 'yes' to confirm or 'no' to cancel."
    )
    update.message.reply_text(summary)
    return CONFIRMATION

def confirm_details(update, context):
    user_response = update.message.text.lower().strip()
    if user_response == 'yes':
        # Generate plan with GPT
        plan_text = generate_gpt_plan(context.user_data)
        # Send the plan text
        update.message.reply_text(plan_text)

        # Attempt to parse the plan for relevant keywords (e.g., place names)
        place_keywords = extract_place_keywords(plan_text)
        if place_keywords:
            update.message.reply_text("Here are other options in the city :")
            city = context.user_data['city']
            for keyword in place_keywords:
                # For each keyword, search with Google Places
                place_info = search_google_place(keyword, city)
                if place_info:
                    place_name, address, photo_bytes, maps_link = place_info
                    # Build a caption with address + clickable link
                    caption = f"*{place_name}*\nAddress: {address}\n[Open in Google Maps]({maps_link})"
                    if photo_bytes:
                        update.message.reply_photo(
                            photo=photo_bytes,
                            caption=caption,
                            parse_mode=ParseMode.MARKDOWN
                        )
                    else:
                        # If no photo, just send text
                        update.message.reply_text(
                            caption,
                            parse_mode=ParseMode.MARKDOWN
                        )
                else:
                    update.message.reply_text(f"Couldn't find place: {keyword}")

        update.message.reply_text("Enjoy your hangout!")
    else:
        update.message.reply_text("No worries. Let's start over with /start if you like.")
    return ConversationHandler.END

def cancel(update, context):
    update.message.reply_text("Conversation canceled. Type /start to begin again.")
    return ConversationHandler.END

# --------------------------------------------------------------------
# 5) GPT HELPER
# --------------------------------------------------------------------
def generate_gpt_plan(user_data):

    prompt = (
        "You are a helpful assistant that plans hangouts. Given the following info:\n\n"
        f"Number of people: {user_data['num_people']}\n"
        f"Names: {', '.join(user_data['names'])}\n"
        f"Ages: {', '.join(user_data['ages'])}\n"
        f"Country: {user_data['country']}\n"
        f"City: {user_data['city']}\n"
        f"Activities: {user_data['activity']}\n"
        f"Time to spend: {user_data['time_spent']}\n"
        f"Preferred time of day: {user_data['time_of_day']}\n"
        f"Budget: {user_data['budget']}\n\n"
        "Provide a short  plan with recommended 3 to 4 places start and end time for each (restaurants, clubs,coffee shops, attractions, etc.).\n"
        "List each place on its own line, starting with a number , e.g.: \n"
        "1) PlaceName\n2) AnotherPlace\n\n"
        "Include specific place names exactly , adress,website ,average cost ."
    )

    try:
        response = client.chat.completions.create(
            model="chatgpt-4o-latest",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=700,
            temperature=0.7
        )
        plan = response.choices[0].message.content.strip()
        return plan
    except Exception as e:
        logger.error(f"OpenAI API error: {e}")
        return "Sorry, I couldn't generate a plan at the moment."

def parse_gpt_place_names(plan_text):

    place_names = []
    pattern = re.compile(r"^(?:[\-*\d]+\)?\s+)(.+)$")

    lines = plan_text.strip().split('\n')
    for line in lines:
        line = line.strip()
        match = pattern.match(line)
        if match:
            place_name = match.group(1).strip()
            if place_name:
                place_names.append(place_name)

    # Remove duplicates if any
    place_names = list(dict.fromkeys(place_names))
    return place_names

def search_places_in_city(query, city, max_results=3):

    text_search_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    full_query = f"{query} in {city}"
    params = {
        "query": full_query,
        "key": GOOGLE_API_KEY
    }

    try:
        resp = requests.get(text_search_url, params=params)
        data = resp.json()
        results = data.get("results", [])
        if not results:
            logger.info(f"No places found for query: {full_query}")
            return []

        # We'll return up to `max_results` with photos
        places_with_photos = []
        for place in results:
            place_name = place.get("name")
            photos = place.get("photos", [])
            if not photos:
                # Skip places with no photo
                continue

            # Get first photo reference
            photo_ref = photos[0].get("photo_reference")
            if photo_ref:
                photo_bytes = get_google_place_photo(photo_ref)
                if photo_bytes:
                    places_with_photos.append((place_name, photo_bytes))

            # Stop if we reached max_results
            if len(places_with_photos) >= max_results:
                break

        return places_with_photos

    except Exception as e:
        logger.error(f"search_places_in_city error: {e}")
        return []

def get_google_place_photo(photo_reference):

    try:
        photo_url = "https://maps.googleapis.com/maps/api/place/photo"
        params = {
            "photoreference": photo_reference,
            "maxwidth": 800,  # or any desired width
            "key": GOOGLE_API_KEY
        }
        resp = requests.get(photo_url, params=params)
        if resp.status_code == 200:
            return resp.content  # raw bytes
        else:
            logger.error(f"Photo request failed, status={resp.status_code}")
            return None
    except Exception as e:
        logger.error(f"get_google_place_photo error: {e}")
        return None
# --------------------------------------------------------------------
# 7) MAIN FUNCTION - TELEGRAM SETUP
# --------------------------------------------------------------------
def main():
    TELEGRAM_BOT_TOKEN = TELEGRAM_BOT_TOKEN1

    updater = Updater(TELEGRAM_BOT_TOKEN, use_context=True)
    dp = updater.dispatcher

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler("start", start)],
        states={
            NUMBER_OF_PEOPLE: [MessageHandler(Filters.text & ~Filters.command, get_number_of_people)],
            GET_NAMES: [MessageHandler(Filters.text & ~Filters.command, get_names)],
            COUNTRY: [MessageHandler(Filters.text & ~Filters.command, get_country)],
            CITY: [MessageHandler(Filters.text & ~Filters.command, get_city)],
            ACTIVITY: [MessageHandler(Filters.text & ~Filters.command, get_activity)],
            TIME_SPENT: [MessageHandler(Filters.text & ~Filters.command, get_time_spent)],
            TIME_OF_DAY: [MessageHandler(Filters.text & ~Filters.command, get_time_of_day)],
            BUDGET: [MessageHandler(Filters.text & ~Filters.command, get_budget)],
            AGES: [MessageHandler(Filters.text & ~Filters.command, get_ages)],
            CONFIRMATION: [MessageHandler(Filters.text & ~Filters.command, confirm_details)]
        },
        fallbacks=[CommandHandler("cancel", cancel)]
    )

    dp.add_handler(conv_handler)

    updater.start_polling()
    updater.idle()

if __name__ == "__main__":
    main()
